In [ ]:
%pip install datasets

In [1]:
import json
import pandas as pd
import os
from sklearn.dummy import DummyClassifier
from datasets import load_dataset

/home/tajak/miniconda3/envs/emma_main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the training dataset
train_df = pd.read_json("../../datasets/EMMediaTopic-training-dataset/EMMediaTopic-1.0.jsonl", orient="records", lines=True)
train_df.head(2)

,document_id,lang,text,GPT-IPTC-label,split
0,CLASSLA-web.hr.4163187,hr,Osnovna škola Pećine Rijeka\n\nObavijest učeni...,education,dev
1,CLASSLA-web.hr.2821678,hr,U organizaciji Lige protiv raka Koprivničko-kr...,health,train


In [4]:
train_df = train_df[train_df["split"] == "train"]
train_df.shape

(20000, 5)

In [5]:
train_df.rename(columns={"GPT-IPTC-label": "labels"}, inplace=True)

train_df.head(2)

,document_id,lang,text,labels,split
1,CLASSLA-web.hr.2821678,hr,U organizaciji Lige protiv raka Koprivničko-kr...,health,train
2,CLASSLA-web.hr.2508108,hr,[VIDEO] Rimac: Ova pobjeda u Beogradu znači pu...,sport,train


In [2]:
# Load the test datasets from the GitHub repositories (access to them is obtained by request to the AGILE repository owner)

iptc_test = pd.read_json("../../datasets/IPTC-top-test/IPTC-top-test.jsonl", lines=True)
print(iptc_test.shape)

iptc_test.rename(columns={"label": "labels"}, inplace=True)

iptc_test.head(2)

(1129, 5)


,document_id,lang,text,labels,multilabel
0,CLASSLA-web.hr.3842863,hr,Iza kamere\n\nFilmsku družinu ZAG posjetila je...,"arts, culture, entertainment and media","[arts, culture, entertainment and media]"
1,CLASSLA-web.hr.4597011,hr,Učenici osmih razreda nedavno su u sklopu tere...,education,[education]


In [8]:
print(iptc_test.groupby("lang").labels.value_counts(normalize=True).to_markdown())

|                                                     |   proportion |
|:----------------------------------------------------|-------------:|
| ('ca', 'society')                                   |    0.108614  |
| ('ca', 'lifestyle and leisure')                     |    0.104869  |
| ('ca', 'economy, business and finance')             |    0.0861423 |
| ('ca', 'arts, culture, entertainment and media')    |    0.082397  |
| ('ca', 'politics')                                  |    0.0749064 |
| ('ca', 'education')                                 |    0.0636704 |
| ('ca', 'labour')                                    |    0.0636704 |
| ('ca', 'environment')                               |    0.0599251 |
| ('ca', 'crime, law and justice')                    |    0.0561798 |
| ('ca', 'science and technology')                    |    0.0561798 |
| ('ca', 'health')                                    |    0.0524345 |
| ('ca', 'sport')                                     |    0.0524345 |
| ('ca

In [9]:
def dummy(train_df, test_df, test_df_name="IPTC-test"):
    # Create X_train and Y_train parts, used for sci kit learning
    # List of texts in training split
    X_train = list(train_df.text)
    # List of labels in training split
    Y_train = list(train_df.labels)

    # List of texts in test split
    X_test = list(test_df.text)
    # List of labels in test split
    Y_test = list(test_df.labels)

    print(len(X_train), len(Y_train), len(X_test), len(Y_test))

    # Create a list of labels
    labels = list(test_df.labels.unique())
    print("Labels: {}".format(labels))

    for strategy in ["stratified", "most_frequent"]:
        model = f"dummy-{strategy}"

        dummy_mf = DummyClassifier(strategy=strategy)

        # Train the model
        dummy_mf.fit(X_train, Y_train)

        #Get the predictions
        y_pred_mf = dummy_mf.predict(X_test)

        y_pred = list(y_pred_mf)

        # Create a json with results
        current_results = {
            "system": model,
            "predictions": [
                {
                "train": "EMMediaTopic (train split)",
                "test": "{}".format(test_df_name),
                "predictions": y_pred,
                }
            ],
            #"model": model_type_dict[model][1],
            #"args": model_args,
            }

        # Save the results as a new json
        with open("submissions/submission-{}-{}.json".format(model, test_df_name), "w") as file:
            json.dump(current_results, file)

        print("Classification with {} on {} finished.".format(model, test_df_name))


In [10]:
dummy(train_df, iptc_test)

20000 20000 1129 1129
Labels: ['arts, culture, entertainment and media', 'education', 'science and technology', 'crime, law and justice', 'society', 'disaster, accident and emergency incident', 'conflict, war and peace', 'labour', 'sport', 'politics', 'economy, business and finance', 'religion', 'lifestyle and leisure', 'health', 'environment', 'human interest', 'weather']
Classification with dummy-stratified on IPTC-test finished.
Classification with dummy-most_frequent on IPTC-test finished.
